# Single Agent on SummEval 

## Imports

In [17]:
import pandas as pd
from datasets import load_dataset
import os
from dotenv import load_dotenv
from autogen import ConversableAgent
import re
from tqdm import tqdm
import numpy as np

## Data

In [27]:
SummEval = load_dataset("mteb/summeval")

df = pd.DataFrame(SummEval["test"])[["text", "machine_summaries", "relevance", "coherence", "fluency", "consistency"]]

problematic_indices = [5, 7, 8, 9, 10, 11, 18, 20, 26, 27, 33, 34, 39, 46, 61, 64, 68, 73, 75, 79, 85, 86, 88, 92, 96, 99]
df_filtered = df.drop(index=problematic_indices).reset_index(drop=True)

df_exploded = df_filtered.explode(["machine_summaries", "relevance", "coherence", "fluency", "consistency"]).reset_index(drop=True)

df_sampled = df_exploded.iloc[:1200].sample(n=100, random_state=42).reset_index(drop=True)

columns_to_round = ["relevance", "coherence", "fluency", "consistency"]

df_sampled[columns_to_round] = df_sampled[columns_to_round].map(lambda x: int(np.ceil(x) if x % 1 > 0.5 else np.floor(x)))

df_final = df_sampled

print(df_final.info())
print(df_final.head(1))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               100 non-null    object
 1   machine_summaries  100 non-null    object
 2   relevance          100 non-null    int64 
 3   coherence          100 non-null    int64 
 4   fluency            100 non-null    int64 
 5   consistency        100 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 4.8+ KB
None
                                                text  \
0  Boss Nigel Pearson has urged Leicester to keep...   

                                   machine_summaries  relevance  coherence  \
0  jamie vardy scored an injury-time winner again...          2          2   

   fluency  consistency  
0        3            2  


## Config

In [30]:
load_dotenv()

api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_API_VERSION", "2023-12-01-preview")

config_list = [
    {
        "model": deployment_name,
        "api_key": api_key,
        "base_url": f"{endpoint}/openai/deployments/{deployment_name}/chat/completions?api-version={api_version}",
        "api_type": "azure",
        "api_version": api_version,  
        "temperature": 0,
        "cache_seed": 42,
    }
]

## System Design

In [31]:
agent_system_message = f"""
You are an objective Evaluator-Agent.
In this task you will evaluate the quality of a summary written for a news article.
To correctly solve this task, follow these steps:

    1. Carefully read the news article, be aware of the information it contains.
    2. Read the proposed summary.
    3. Rate the summary with integer values on a scale from 1 (worst) to 5 (best) by its relevance, consistency, fluency, and coherence.

Definitions:
    Relevance:
        - The rating measures how well the summary captures the key points of the article.
        - Consider whether all and only the important aspects are contained in the summary.
    Consistency:
        - The rating measures whether the facts in the summary are consistent with the facts in the original article.
        - Consider whether the summary does reproduce all facts accurately and does not make up untrue information.
    Fluency:
        - This rating measures the quality of individual sentences, are they well-written and grammatically correct.
        - Consider the quality of individual sentences.
    Coherence:
        - The rating measures the quality of all sentences collectively, to the fit together and sound naturally.
        - Consider the quality of the summary as a whole

Give an explanation on your decisions using about 200 words.
Always begin your output with: "As an objective Evaluator-Agent I think ..."
Always end your output with a JSON object with the following format:{{"relevance": score, "coherence": score, "fluency": score, "consistency": score}} 
"""

In [32]:
initializer = ConversableAgent(
    "initializer", 
    llm_config={"config_list": config_list},
    human_input_mode="NEVER",
    )

agent = ConversableAgent(
    "Evaluator-Agent", 
    llm_config={"config_list": config_list},
    system_message=agent_system_message,
    human_input_mode="NEVER",
    )

## Evaluation

In [33]:
def evaluate(text, machine_summaries, relevance, coherence, fluency, consistency):
    message = f""" 
    Article: {text}

    Summary: {machine_summaries}
    """

    result = initializer.initiate_chat(agent, message=message, max_turns=1)
    result_str = str(result)
    print(result_str)

    pattern = r'"relevance"\s*:\s*(\d+)'
    relevance_score = int(re.search(pattern, result_str).group(1))

    pattern = r'"coherence"\s*:\s*(\d+)'
    coherence_score = int(re.search(pattern, result_str).group(1))

    pattern = r'"fluency"\s*:\s*(\d+)'
    fluency_score = int(re.search(pattern, result_str).group(1))

    pattern = r'"consistency"\s*:\s*(\d+)'
    consistency_score = int(re.search(pattern, result_str).group(1))

    relevance_deviation = abs(relevance - relevance_score) 
    coherence_deviation = abs(coherence - coherence_score) 
    fluency_deviation = abs(fluency - fluency_score) 
    consistency_deviation = abs(consistency - consistency_score) 

    return {
        "relevance": {
            "ground_truth": relevance,
            "system_decision": relevance_score,
            "deviation": relevance_deviation
        },
        "coherence": {
            "ground_truth": coherence,
            "system_decision": coherence_score,
            "deviation": coherence_deviation
        },
        "fluency": {
            "ground_truth": fluency,
            "system_decision": fluency_score,
            "deviation": fluency_deviation
        },
        "consistency": {
            "ground_truth": consistency,
            "system_decision": consistency_score,
            "deviation": consistency_deviation
        }
    }

In [34]:
num_rows = 100

results = []

df_subset = df_final.head(num_rows)

for _, row in tqdm(df_subset.iterrows(), total=num_rows, desc="Progress"):
    result = evaluate(
        text=row["text"],
        machine_summaries=row["machine_summaries"],
        relevance=row["relevance"],
        coherence=row["coherence"],
        fluency=row["fluency"],
        consistency=row["consistency"]
    )
    results.append(result)

results_df = pd.DataFrame(results)
results_df.to_csv('Results/single.csv', index=False)

Progress:   0%|          | 0/100 [00:00<?, ?it/s]

initializer (to Evaluator-Agent):

 
    Article: Boss Nigel Pearson has urged Leicester to keep their cool and ignore their relegation rivals. The Foxes host Swansea on Saturday just three points from safety in the Barclays Premier League after back-to-back wins. Last week's 3-2 win at West Brom handed them a survival lifeline, although they remain bottom of the table. Jamie Vardy scored an injury-time winner against West Bromwich Albion on Saturday to improve his side's slim chance of Premier League survival Vardy celebrates in front of the travelling away fans after hitting the winner against West Brom But after their mini-revival, Pearson wants his side to remain focused on their own jobs. 'I'm very wary of people flipping the emphasis,' he said. 'Our future is in our own hands and if we go into the last game with that we have given ourselves a realistic chance. 'We need to make sure our own run-in is what we want it to be. Leicester manager Nigel Pearson has urged his players to f

Progress:   1%|          | 1/100 [00:03<05:03,  3.06s/it]

ChatResult(chat_id=None, chat_history=[{'content': " \n    Article: Boss Nigel Pearson has urged Leicester to keep their cool and ignore their relegation rivals. The Foxes host Swansea on Saturday just three points from safety in the Barclays Premier League after back-to-back wins. Last week's 3-2 win at West Brom handed them a survival lifeline, although they remain bottom of the table. Jamie Vardy scored an injury-time winner against West Bromwich Albion on Saturday to improve his side's slim chance of Premier League survival Vardy celebrates in front of the travelling away fans after hitting the winner against West Brom But after their mini-revival, Pearson wants his side to remain focused on their own jobs. 'I'm very wary of people flipping the emphasis,' he said. 'Our future is in our own hands and if we go into the last game with that we have given ourselves a realistic chance. 'We need to make sure our own run-in is what we want it to be. Leicester manager Nigel Pearson has urge

Progress:   2%|▏         | 2/100 [00:05<04:14,  2.59s/it]

...

ChatResult(chat_id=None, chat_history=[{'content': " \n    Article: These extraordinary images show how an enormous jumbo jet has been converted into a luxury home for a mystery billionaire - complete with bedrooms, multiple lounges and an onboard restaurant. The custom-built Boeing 747 is believed to have cost its ultra-wealthy owner £400million after it was customised to fit his specifications exactly over a three-year period. The jumbo jet would normally carry as many as 600 passengers - but this version was built for just a single tycoon, although he will be able to fit dozens of guests. It contains a large dining room where travellers can enjoy meals cooked in an adjoining kitchen, with a lounge and 'state room' at the back of the plane. More sleeping space is provided in the 'aeroloft' on the top deck of the vessel, with eight full beds for passengers who prefer to get some shut-eye on flights. Luxury: This is the lounge on board a custom-built Boeing 747-8, as depicted in an art

Progress:  99%|█████████▉| 99/100 [03:48<00:03,  3.05s/it]

ChatResult(chat_id=None, chat_history=[{'content': " \n    Article: Barcelona club president Josep Maria Bartomeu has insisted that the La Liga leaders have no plans to replace Luis Enrique and they're 'very happy' with him. The 44-year-old took only took charge of the club last summer, signing a two-year-deal, and is only six games away from winning the title. Despite speculation this season that Enrique will be replaced in the summer, Bartomeu refuted these claims and says he's impressed with how the manager has performed. Luis Enrique only took charge at the club last summer and has impressed during his tenure Barcelona president Josep Maria Bartemou says the club are 'very happy' with Enrique's performance 'There's a lot of time to talk about his contract in the future,' the Barca president told bySPORT. 'Luis Enrique is concentrating on La Liga and on everything he needs to do in relation to the team. 'After the Champions League game against PSG in Paris on Wednesday, I saw how ha

Progress: 100%|██████████| 100/100 [03:52<00:00,  2.32s/it]

ChatResult(chat_id=None, chat_history=[{'content': " \n    Article: Team-mates Neymar and Dani Alves proved their dedication to Barcelona by supporting the club’s basketball side. Neymar and Alves headed to watch El Clasico on Thursday night alongside the Brazilian's sister Rafaella. Barca prevailed with a narrow 85-80 victory in the Euro League contest. Brazil star Neymar (centre) takes a selfie with friends and Barcelona team-mate Dani Alves (right) However Real Madrid remain top of their Euro League division over their bitter rivals, just by points difference. Neymar helped Brazil beat Chile 1-0 at the Emirates Stadium on Sunday in a feisty contest and had to withstand several brutal challenges from the South American rivals. Before the international break Luis Enrique's Barcelona had won their El Clasico contest to move four points clear at the top of La Liga. Neymar and his Barcelona team-mates return to La Liga on Sunday, taking on Celta Vigo as they continue to compete for a tre